# Analyzing videos of single lines

## Initialization

In [ ]:
%load_ext autoreload
%autoreload 2
LOGGERDEFINED = False

In [ ]:
import os
import sys
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)
sys.path.append(os.path.dirname(module_path))
from py.config import cfg
import py.logs as logs
import py.vidTools as vt
from py.imshow import imshow
import matplotlib.pyplot as plt
LOGGERDEFINED = logs.openLog('vidAnalysis.ipynb', LOGGERDEFINED, level='DEBUG', exportLog=False)

# measure videos

measure all videos

### measure all videos

In [ ]:
# measure all of the videos
errorVids = vt.measureVideosRecursiveSingle(os.path.join(cfg.path.server, r'singleLines\vels LapRD LapRD 1day\I_3.00_S_3.00'), \
                                      diag=0, overwrite=0, measureHoriz=True, measureXS=True, exportVert=True, exportHoriz=True)

### summarize all video data

In [ ]:
tt, u = vt.videoSummarySingle(os.path.join(cfg.path.server, 'singleLines'), cfg.path.fig)

In [ ]:
import metrics as me
vs,uv = me.importStillsSummary(file='videoSummary.csv', diag=True)

### redo bad measurements

In [ ]:
# badfiles = vs[(vs.projectionN.isna())|(vs.projectionN_N<5)]
badfiles = vs[(vs.sweepType=='visc_20')&(vs.vertDispBotN>0.5)]
badfiles[['bn', 'projectionN']]

In [ ]:
# redo all of the measurements for badfiles
for i,row in badfiles.iterrows():
    vd = vt.vidData(row['folder'])
    vd.detectNozzle()
    vd.measureVideoHoriz(overwrite=1, topthresh=120)

# test one video

In [ ]:
# initialize the detection program
# folder = badfiles.iloc[1]['folder']
folder = os.path.join(cfg.path.server, 'singleLines', r'vels LapRD LapRD 1day\I_3.00_S_3.00\I_3.00_S_3.00_VI_7.5_VS_5_211109')
vd = vt.vidData(folder, pxpmm=139)
vd.detectNozzle()

### image series

In [ ]:
folder = os.path.join(cfg.path.server, 'singleLines', r'vels mineral LapRD\I_M5_S_3.00\I_M5_S_3.00_VI_2.5_VS_5_211005')
vd = vt.vidData(folder, pxpmm=139)
dt = 0.05
fig = vd.showFrames([41+dt*i for i in range(8)], 
              crop={'x0':300, 'xf':-1, 'y0':0, 'yf':-1},
             figw=6.5)
fig.savefig(os.path.join(folder, 'time_series.svg'), bbox_inches='tight', dpi=300)

In [ ]:
folder = os.path.join(cfg.path.server, 'singleLines', r'vels mineral LapRD\I_M6_S_3.00\I_M6_S_3.00_VI_2.5_VS_5_210824')
vd = vt.vidData(folder, pxpmm=139)
dt = 0.25
fig = vd.showFrames([41+dt*i for i in range(8)], 
              crop={'x0':300, 'xf':-1, 'y0':0, 'yf':-1},
             figw=6.5)
fig.savefig(os.path.join(folder, 'time_series.svg'), bbox_inches='tight', dpi=300)

#### fix nozzle detection

In [ ]:
# show nozzle dimensions
vd.xL, vd.xR, vd.yB

In [ ]:
# draw nozzle dimensions on the image
# vd.yB = 420
vd.drawDiagnostics(1)

In [ ]:
# export the nozzle dimensions
vd.exportNozzleDims()

In [ ]:
# manually change the nozzle mask
vd.xL = 320
vd.xR = 433
vd.yB = 340
vd.createNozzleMask()

In [ ]:
# measure a single frame
vd.measureFrameFromLine('vert1', 0.62, diag=2, topthresh=130)

In [ ]:
# measure horizontal lines
vd.measureVideoHoriz(overwrite=1, topthresh=130)

#### overwrite images

In [ ]:
# show a lineup of frames
dt0 = -1.5
flist = [vd.getLineFrame('xs2', t) for t in np.arange(dt0, dt0+1.1, 0.1)]
imshow(*flist)

In [ ]:
# overwrite the still frame
vd.overwriteFrame('xs2',-1.06, diag=True)

In [ ]:
# measure XS metrics
vd.measureVideoXS(overwrite=1, diag=1)

## test nozzle detection over several videos

In [ ]:
run ../tests/test_nozDetect